# 鱼佬-2019CCF-BDCI-乘用车细分市场销量预测方案（Top1%）
https://zhuanlan.zhihu.com/p/93602770

## 评测指标
在我看来对评测指标的理解，也是能够帮助上分的。

初赛复赛阶段的在线评分采用NRMSE（归一化均方根误差）的均值作为评估指标。首先单独计算每个车型在每个细分市场（省份）的NRMSE，再计算所有NRMSE的均值，计算方式为：

不难理解这里需要按车型在不同省份的得分，然后整合起来得到最终的分数。

关键来了，这样的评价指标有一个特点，销量小的车型对评分带来更大的影响，依据这个特征，可以考虑添加样本权重，以及最终融合方式。

下面给出了计算权重的方式，当然也是出于经验的方式构造的，可以有更多优化。

In [1]:
# 样本权重信息
data['n_salesVolume'] = np.log(data['salesVolume']+1)
df_wei = data.groupby(['province','model'])['n_salesVolume'].agg({'mean'}).reset_index().sort_values('mean')
df_wei.columns = ['province','model','wei']
df_wei['wei'] = 10 - df_wei['wei'].values
data = data.merge(df_wei, on=['province','model'], how='left')

只需在模型训练的时候加上权重信息即可，初赛有千分位的提升，复赛没有具体测。

In [ ]:
dtrain = lgb.Dataset(df[all_idx][features], label=df[all_idx]['n_label'], weight=df[all_idx]['wei'].values)

下面给出评价指标的代码：

In [ ]:
def score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

## 特征工程
![](imgs/cycyl01.jpg)

# 一等奖团队分享 | 2019 CCF BDCI 《乘用车细分市场销量预测》 

赛题直播｜《乘用车细分市场销量预测方案分享》赛题方案讲解    
https://mp.weixin.qq.com/s/13s1JG_HSPyjtbp1TL1Nbg  

赛题介绍

比赛链接：https://www.datafountain.cn/competitions/352

技术方向：预测回归，数据挖掘

赛题背景：

深瞳是一家大数据与行业智能应用解决方案运营商，为各行业客户提供数   据分析与策略咨询服务，帮助行业客户进行数据资产化，为客户提供数 处理、建模分析服务。汽车行业是深瞳所重点服务的核心行业之一，长期服务于国内外知名汽车品牌客户。

近几年来，国内汽车市场由增量市场逐步进入存量市场阶段，2018年整   体市场销量首次同比下降。	在市场整体趋势逐步改变的环境下，消费者购车决策的过程也正在从线下向线上转移，我们希望能在销量数据自身趋势规律的基础上，找到消费者在互联网上的行为数据与销量之间的相关性，为汽车行业带来更准确有效的销量趋势预测。

赛题任务：

基于该模型预测同一款车型和相同细分市场在接下来一个季度连续4个月份的销量；除销量数据外，还提供同时期的用户互联网行为统计数据，包括：各细分市场每个车型名称的互联网搜索量数据；主流汽车垂直媒体用户活跃数据等。

参赛队伍可同时使用这些非销量数据用于建模。除了模型的准确性外，参赛队伍需对本赛题任务有系统性的思考和设计，在决赛阶段，参赛队伍对于所提交的模型的适应性、可扩展性、代码的工程性等方面也会影响参赛队伍的最终名次。



https://mp.weixin.qq.com/s?__biz=MzI5ODQxMTk5MQ==&mid=2247487049&idx=1&sn=896da47626d37e9ae34611a296cb1bb6&chksm=eca77cf1dbd0f5e725b015dc150d3b517303ae124ee37ab6e0e5872dcbc1ca3f58ca683e2526&scene=21#wechat_redirect

https://zhuanlan.zhihu.com/p/98926322

讲者介绍


梁晨
计算机科学与技术研究生在读
赛题《乘用车细分市场销量预测》一等奖团队队长

代码


我们本次的方案一共约500行代码，主要的工作在于特征工程与规则构造，思路，代码都很简单，运行只需3min，请放心食用。


## 开源源码：
https://github.com/cxq80803716/2019-CCF-BDCI-Car_sales

## 赛题解读
本赛题需要参赛队伍根据给出的60款车型在22个细分市场（省份）的销量连续24个月（从2016年1月至2018年12月）的销量数据，建立销量预测模型；基于该模型预测同一款车型和相同细分市场在接下来一个季度连续4个月份的销量；除销量数据外，还提供同时期的用户互联网行为统计数据，包括：各细分市场每个车型名称的互联网搜索量数据；主流汽车垂直媒体用户活跃数据等。参赛队伍可同时使用这些非销量数据用于建模。

简单来说，本次赛题给出2016.1~2017.12的省份，车型，车身，销量，搜索量，评论量，评价量等，要求预测2018.1~2018.4的销量。

评价指标是归一化均方根误差的均值
![](imgs/cyc20.png)

## 原始数据
通过初步分析数据可以发现省份-车型所组成的类别特别多，并且对于同一车型，波动也挺大的，销量的范围较大等。

顺便说一下，我们经过多次尝试发现评论量与评价量几乎起不到作用，所以这两个特征我们并没有使用，如果你有办法处理这两个特征，欢迎评论讨论。

![](imgs/cyc21.png)

## 数据预处理

为了使数据分布更加符合高斯分布，对数据做了log1p的预处理。
![](imgs/cyc22.png)

## 算法整体方案
我们在初赛的时候尝试了xgb,lgb,cat,prophet,rule,lstm,cnn，并且初赛的最终结果是由lgb,prophet,rule与lstm融合而来。但是后来我们发现就算只用lgb和rule也能得到差不多的分数，时序模型与深度学习模型在这道小数据时序问题上并不适用，又考虑到工业环境中模型越少，越简单越好，因此复赛时，我们只使用了差异性足够大的lgb与rule两个模型。

![](imgs/cyc01.png)

## 数据探索EDA

在构造特征之前，对特征一些分析。

画出省份销量的热力图可以发现销量与省份的发达程度，临海程度是成一定正相关的，这与我们的直觉相符。因此各省份销量的差异明显，建模时必须对省份进行区别。

对于同一个月份，2016年与2017年的销量类似，即销量有周期性变化的特性。而且对于宏观变化而言，2017年的销量较2016年销量呈现下降趋势。


春节月的销量较低，春节后逐渐上升，年末达到最高。
![](imgs/cyc03.png)

## 特征工程

这一块是本次工作的重头戏，也是我们花费了最多时间的地方，最终，我们的模型特征由以下几部分组成。


* 考虑到春节等节假日，我们构造了与节假日相关的一些特征。
* 考虑到每个月的天数，工作日不同，我们构造了相关的特征。
* 由于是个时序问题，因此历史销量与销量的变化趋势是我们应该考虑的重点。针对此，我们在多个不同粒度下构造了历史平移特征，差分特征，同比/环比特征与趋势特征。

不过经过我们的尝试，同比的效果不怎么好。

更加详细的内容请看代码。
![](imgs/cyc04.png)

## 特征选择


我们所构造的特征实际上不止这些，不过由于信息重叠等原因，我们使用了一些方法对特征进行筛选。

我们使用了树模型的特征重要性排序，均值判断与SHAP进行特征筛选。

* 特征重要性排序：根据树模型输出的特征重要性进行筛选，去掉重要性低的特征。

* 均值判断：由于本道赛题中，1234月具有相对固定的均值比例与均值大小，因此可以通过添加/删除特征后1234月的平均均值来大概判断特征的好坏。

* SHAP：利用了组合博弈论的知识，防止因为信息重叠而导致的特征重要度不公平的情况。
![](imgs/cyc05.png)

## 数据划分
抑制误差传递


直接使用鱼佬的框架能带来相当不错的效果，但是可以发现，越往后的月份误差的累计会越来越严重，因此在模型中，我们只拼接了1月份预测出来的值。(带来了0.01的精度提升)
![](imgs/cyc02.png)

## 初赛LGB的构建流程


初赛时lgb做的一些操作与对应的分数如下
![](imgs/cyc06.png)

## 统计规则


规则部分的框架是使用历史同期销量，历史最近销量与指数平滑进行加权得到一个初步预测销量，然后用上下半年的销量变化趋势与平滑构造年度销量变化趋势，最后两者相乘即可得到规则的预测结果。


由此可见，规则带有相当多的超参数。据我了解不少团队复赛时规则血崩，这也是时序题里面规则的泛性问题。我们初赛时规则可以达到0.633，是一个绝对的主力，复赛提交次数太少，最终规则也只有0.598，只能以lgb为主，规则为辅。
![](imgs/cyc07.png)
![](imgs/cyc08.png)
![](imgs/cyc09.png)
![](imgs/cyc10.png)

## 模型融合


因为只有两个模型，所以模型融合基本上不用考虑太多，直接进行简单的算数/几何加权即可，由于几何加权可以使预测值偏小，而2018年的销量理应是较之前低的，所以我们使用了几何加权进行融合。


另外1234月分开进行融合，可以带来微小的提升。
![](imgs/cyc11.png)

## 总结


这里特别提一下模型的数量与运行时间的优点，据我们决赛观察，大多数队伍都有超过2个的模型，而且有的队伍需要超过一个小时的运行时间。从工业角度来讲，我们的方案应该是更加适用的。

不过还有很多可以提升的地方，比如评论，评价的使用方式，规则中超参数过多问题，从某个时间点开始的某项政策对后续的影响等。如果赛题能提供更多的特征与数据，相信可做的地方还有相当多。
![](imgs/cyc12.png)